# 2. NHITS/NBEATS Quick Start

<a href="https://colab.research.google.com/github/Nixtla/neuralforecast/blob/main/nbs/examples/Getting_Started_with_NBEATS_and_NHITS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing NeuralForecast Library

In [ ]:
%%capture
!pip install neuralforecast

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display, Markdown

import matplotlib.pyplot as plt
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, NHITS
from neuralforecast.utils import AirPassengersDF

## Loading AirPassengers Example Data

In [ ]:
# We define the train df. 
Y_df = AirPassengersDF
Y_df.head()

## Fit NBEATS and NHITS

### Split train/test sets

In [ ]:
Y_train_df = Y_df[Y_df.ds<='1959-12-31'] # 132 train
Y_test_df = Y_df[Y_df.ds>'1959-12-31'] # 12 test

### Fit the models

Using NeuralForecast you can fit a set of models to your dataset. You just have to define the `input_size` and `horizon` of your model. The `input_size` is the number of historic observations that the model will use to learn to predict `h` steps in the future. Also, you can modify the hyperparameters of the model to get a better accuracy.

In [ ]:
%%capture
horizon = len(Y_test_df)
models = [
    NBEATS(input_size=2 * horizon, h=horizon, max_epochs=100),
    NHITS(input_size=2 * horizon, h=horizon, max_epochs=100)
]
model = NeuralForecast(
    models=models,
    freq='M', 
)
model.fit(df=Y_train_df)

### Predict using the fitted models

In [ ]:
Y_hat_df = model.predict(horizon).reset_index()
Y_hat_df.head()

## Plot and Evaluate Predictions

We are going to plot the models againts the real values of test.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (20, 7))
Y_hat_df = Y_test_df.merge(Y_hat_df, how='left', on=['unique_id', 'ds'])
plot_df = pd.concat([Y_train_df, Y_hat_df]).set_index('ds')

plot_df[['y', 'NBEATS', 'NHITS']].plot(ax=ax, linewidth=2)

ax.set_title('AirPassengers Forecast', fontsize=22)
ax.set_ylabel('Monthly Passengers', fontsize=20)
ax.set_xlabel('Timestamp [t]', fontsize=20)
ax.legend(prop={'size': 15})
ax.grid()

Finally, we evaluate the predictions accuracy using the Mean Absolute Error:

$$
\qquad MAE = \frac{1}{Horizon} \sum_{\tau} |y_{\tau} - \hat{y}_{\tau}|\qquad
$$

In [ ]:
def mae(y_hat, y_true):
    return np.mean(np.abs(y_hat-y_true))

y_true = Y_test_df['y'].values
nbeats_preds = Y_hat_df['NBEATS'].values
nhits_preds = Y_hat_df['NHITS'].values

print('NBEATS    MAE: %0.3f' % mae(nbeats_preds, y_true))
print('NHITS     MAE: %0.3f' % mae(nhits_preds, y_true))
print('ETS       MAE: 16.222')
print('AutoARIMA MAE: 18.551')

You can find [here](https://nixtla.github.io/statsforecast/examples/getting_started_with_auto_arima_and_ets.html) the reproduction of the ETS and AutoARIMA performances.

<a href="https://colab.research.google.com/github/Nixtla/neuralforecast/blob/main/nbs/examples/Getting_Started_with_NBEATS_and_NHITS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>